# English - Italian Embeddings (3 versions)
`w266 Final Project: Crosslingual Word Embeddings`

Instead of traning on randomly substituted words, here we'll choose the translation that is closest to the context embedding vector.

# Notebook Setup

In [4]:
# general imports
from __future__ import print_function
import time
import pickle
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

# tell matplotlib not to open a new window
%matplotlib inline

# autoreload modules
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


__Base Paths__

In [79]:
BASE = '/home/mmillervedam/Data'
PROJ = '/home/mmillervedam/ProjectRepo'
GTT_BASE = PROJ + '/BaselineModels/data/ground_truth_translations/'
#PROJ = '/Users/mona/OneDrive/repos/final_proj/W266-Fall-2017-Final-Project'

# directory to save pickled embeddings
SAVE_TO = BASE + '/embeddings'

__Globals__ - _the parameters below fully determine all 3 models in this NB_

In [116]:
# Data
LANG = ('en','it')
FULL_TEXT = "/home/miwamoto/en_it_shuf.txt"
VOCAB_INDEX = BASE + '/vocab/en_it_small.pkl'
PANLEX = BASE + '/panlex/en_it_dict.pkl'
GTT_PATH = GTT_BASE + "%s-%s-clean.csv" % (LANG[0], LANG[1])

# Model
EMBEDDING_SIZE = 200

# Training
nBATCHES = 50000 # <<< 1 epoch with our 1 million sentence corpus
BATCH_SIZE = 48
WINDOW_SIZE = 4
MAX_EPOCHS = 5 # fail safe
ALPHA = 0.5 # authors use a much smaller learning rate but train longer

# Load Data

In [102]:
from parsing import Corpus, BilingualVocabulary, batch_generator, get_common_words

In [64]:
# load corpus
raw_data = Corpus(FULL_TEXT)

In [65]:
# load panlex dictionary
with open(PANLEX,'rb') as f:
    translations = pickle.load(f)

In [66]:
# load vocabulary
vocab = BilingualVocabulary([], languages = LANG)
with open(VOCAB_INDEX,'rb') as f:
    vocab.load_from_index(pickle.load(f))

In [67]:
# confirmations
print('... loaded %s panlex translations'%(len(translations)))
print('... loaded %s word %s vocabulary'%(vocab.size,vocab.language))

... loaded 525091 panlex translations
... loaded 20003 word ('en', 'it') vocabulary


In [77]:
# Validation Words (for training printout)
TEST_WORDS = vocab.to_ids(['en_the','en_last', 'it_si', 'it_suo'])
print('... test word ids:', TEST_WORDS)

... test word ids: [3, 239, 10020, 10040]


In [117]:
# Ground Truth Translations
GTT_DF = pd.read_csv(GTT_PATH, names = [LANG[0], LANG[1]], sep=' ', header=None)
print('... loaded %s ground truth translations.'%(len(GTT_DF)))

... loaded 103613 ground truth translations.


In [118]:
a = get_common_words(vocab)

In [119]:
a

{'en_comics',
 'en_basel',
 'en_counterpart',
 'it_migliora',
 'en_things',
 'en_attested',
 'en_karnataka',
 'it_migliore',
 'en_expenses',
 'it_ritmica',
 'en_masterpiece',
 'en_state',
 'en_distortion',
 'en_killers',
 'it_platone',
 'en_disk',
 'it_lentamente',
 'it_promettente',
 'it_consolidare',
 'en_bengali',
 'en_interim',
 'en_inevitable',
 'en_examination',
 'en_ethics',
 'it_precisione',
 'it_foto',
 'en_extended',
 'it_queen',
 'en_southwest',
 'it_sonata',
 'en_wake',
 'en_crash',
 'en_transformation',
 'en_exact',
 'it_ginevra',
 'it_sino',
 'it_apre',
 'en_get',
 'en_oceans',
 'en_gear',
 'it_spartiacque',
 'it_consumo',
 'en_complex',
 'it_vegetazione',
 'it_limitare',
 'it_cattura',
 'en_storms',
 'en_ran',
 'en_czechoslovakia',
 'en_saint',
 'en_bmw',
 'en_develop',
 'en_timing',
 'en_initiation',
 'it_sorprendente',
 'it_cuori',
 'it_improvviso',
 'en_bottom',
 'it_improvvisa',
 'en_mall',
 'it_cuore',
 'it_mostra',
 'en_stefan',
 'it_space',
 'it_morente',
 'en_beg

In [120]:
# Evaluation Words (for reporting recall)
eval_words = [w for w in get_common_words(vocab) if w.startswith(LANG[1])]
EVAL_IDS = vocab.to_ids(eval_words)
print('... loaded %s evaluation words.' % (len(EVAL_IDS)))

... loaded 8516 evaluation words.


# Method 1: Random Translations

### Initialize the model

In [39]:
from models import BiW2V_random

# create model
model_1 = BiW2V_random(bilingual_dict = translations,
                       vocab = vocab, H = EMBEDDING_SIZE)

# intialize TF graphs
model_1.BuildCoreGraph()
model_1.BuildTrainingGraph()
model_1.BuildValidationGraph()

... TF graph created for BiW2V model.
... TF graph created for BiW2V training.
... TF graph created for BiW2V validation.


### Training

In [40]:
# fresh data generator
DATA_GENERATOR = batch_generator(raw_data, vocab, BATCH_SIZE, WINDOW_SIZE, MAX_EPOCHS)

In [41]:
# train
start = time.time()
model_1.train(nBATCHES, DATA_GENERATOR, TEST_WORDS, learning_rate = ALPHA)
tot = (time.time() - start)
print('... {} batches trained in {} seconds'.format(nBATCHES, tot))

... Model Initialized
	 <tf.Variable 'Embedding_Layer/ContextEmbeddings:0' shape=(20003, 200) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/WordEmbeddings:0' shape=(20003, 200) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/b:0' shape=(20003,) dtype=float32_ref>
... Starting Training
... STEP 0 : Average Loss : 0.000264484786987
   [en_the] closest:  it_cristina, en_commented, it_sacerdotale, it_amiga, it_vantaggio, en_muhammad, it_gemello, it_esistente,
   [en_last] closest:  en_n, en_ridge, it_kg, it_cancellare, it_professionistico, en_quad, it_consentire, it_arti,
   [it_si] closest:  it_compiti, it_limitazione, en_sculptor, en_switching, en_solidarity, en_hereford, it_u, it_pesante,
   [it_suo] closest:  it_abitanti, it_presentarsi, en_convert, en_regulate, it_notazione, en_son-in-law, it_compresa, it_girone,
... STEP 50000 : Average Loss : 4.10927392767
... STEP 100000 : Average Loss : 3.85516811703
   [en_the] closest:  en_a, en_its, en_an, en_one, en_his, it_la, it_riunirsi, 

### Save the Embeddings.

In [43]:
# context 
filename = SAVE_TO + '/en_it_rand_500K_V_dec19.pkl'
with open(filename, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(model_1.context_embeddings, f, pickle.HIGHEST_PROTOCOL)

# word
filename = SAVE_TO + '/en_it_rand_500K_U_dec19.pkl'
with open(filename, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(model_1.word_embeddings, f, pickle.HIGHEST_PROTOCOL)

### Evaluation

In [101]:
model_1.evaluate(source_lang = LANG[0], 
                 target_lang = LANG[1], 
                 gtt = GTT_DF, 
                 sample = EVAL_IDS,
                 top_k = 5,
                 verbose = True)

... Model Initialized
sim shape (8516, 20003)
vocab size 20003
word it_migliora
half of vocab 10003
word it_migliore
half of vocab 10004
word it_ritmica
half of vocab 10005
word it_platone
half of vocab 10006
word it_lentamente
half of vocab 10007
word it_promettente
half of vocab 10008
word it_consolidare
half of vocab 10009
word it_precisione
half of vocab 10010
word it_foto
half of vocab 10011
word it_queen
half of vocab 10012
word it_sonata
half of vocab 10013
word it_ginevra
half of vocab 10014
word it_sino
half of vocab 10015
word it_apre
half of vocab 10016
word it_spartiacque
half of vocab 10017
word it_consumo
half of vocab 10018
word it_vegetazione
half of vocab 10019
word it_limitare
half of vocab 10020
word it_cattura
half of vocab 10021
word it_sorprendente
half of vocab 10022
word it_cuori
half of vocab 10023
word it_improvviso
half of vocab 10024
word it_improvvisa
half of vocab 10025
word it_cuore
half of vocab 10026
word it_mostra
half of vocab 10027
word it_space
half

word it_suicida
half of vocab 10230
word it_dos
half of vocab 10231
word it_triassico
half of vocab 10232
word it_savona
half of vocab 10233
word it_anale
half of vocab 10234
word it_cina
half of vocab 10235
word it_costi
half of vocab 10236
word it_guardare
half of vocab 10237
word it_master
half of vocab 10238
word it_osservatori
half of vocab 10239
word it_veneti
half of vocab 10240
word it_veneto
half of vocab 10241
word it_stabile
half of vocab 10242
word it_proprietà
half of vocab 10243
word it_lezione
half of vocab 10244
word it_opzione
half of vocab 10245
word it_journal
half of vocab 10246
word it_clan
half of vocab 10247
word it_dotazione
half of vocab 10248
word it_denti
half of vocab 10249
word it_clay
half of vocab 10250
word it_settimanale
half of vocab 10251
word it_video
half of vocab 10252
word it_lubiana
half of vocab 10253
word it_dente
half of vocab 10254
word it_justice
half of vocab 10255
word it_rettangolare
half of vocab 10256
word it_saccheggio
half of vocab 10

word it_inizi
half of vocab 10457
word it_predecessore
half of vocab 10458
word it_incontri
half of vocab 10459
word it_perfetta
half of vocab 10460
word it_riunito
half of vocab 10461
word it_incontra
half of vocab 10462
word it_precedenza
half of vocab 10463
word it_emerso
half of vocab 10464
word it_marginale
half of vocab 10465
word it_gola
half of vocab 10466
word it_circolo
half of vocab 10467
word it_riposo
half of vocab 10468
word it_sorella
half of vocab 10469
word it_sorelle
half of vocab 10470
word it_carica
half of vocab 10471
word it_lirico
half of vocab 10472
word it_conosciuto
half of vocab 10473
word it_elevato
half of vocab 10474
word it_flashback
half of vocab 10475
word it_ultimi
half of vocab 10476
word it_scorrere
half of vocab 10477
word it_parere
half of vocab 10478
word it_nazionale
half of vocab 10479
word it_confusione
half of vocab 10480
word it_battuta
half of vocab 10481
word it_temperamento
half of vocab 10482
word it_edizione
half of vocab 10483
word it_f

word it_notizie
half of vocab 10681
word it_maschi
half of vocab 10682
word it_rimanenti
half of vocab 10683
word it_teatro
half of vocab 10684
word it_messico
half of vocab 10685
word it_douglas
half of vocab 10686
word it_rimanente
half of vocab 10687
word it_caserma
half of vocab 10688
word it_esistenti
half of vocab 10689
word it_decisiva
half of vocab 10690
word it_albert
half of vocab 10691
word it_ungheresi
half of vocab 10692
word it_elfi
half of vocab 10693
word it_morso
half of vocab 10694
word it_ungherese
half of vocab 10695
word it_bicchiere
half of vocab 10696
word it_visuale
half of vocab 10697
word it_scritto
half of vocab 10698
word it_verona
half of vocab 10699
word it_piave
half of vocab 10700
word it_motocicletta
half of vocab 10701
word it_predatori
half of vocab 10702
word it_blues
half of vocab 10703
word it_permanenza
half of vocab 10704
word it_jordan
half of vocab 10705
word it_radici
half of vocab 10706
word it_impegnato
half of vocab 10707
word it_poesia
hal

word it_one
half of vocab 10911
word it_vetro
half of vocab 10912
word it_onu
half of vocab 10913
word it_geologia
half of vocab 10914
word it_improvvisamente
half of vocab 10915
word it_rappresaglia
half of vocab 10916
word it_quinta
half of vocab 10917
word it_stato
half of vocab 10918
word it_manifestare
half of vocab 10919
word it_mite
half of vocab 10920
word it_mito
half of vocab 10921
word it_amante
half of vocab 10922
word it_hobbit
half of vocab 10923
word it_tragedia
half of vocab 10924
word it_vestibolo
half of vocab 10925
word it_vecchi
half of vocab 10926
word it_minerali
half of vocab 10927
word it_grosso
half of vocab 10928
word it_hamilton
half of vocab 10929
word it_adattamento
half of vocab 10930
word it_sita
half of vocab 10931
word it_camillo
half of vocab 10932
word it_sith
half of vocab 10933
word it_percorso
half of vocab 10934
word it_dominare
half of vocab 10935
word it_compleanno
half of vocab 10936
word it_perugia
half of vocab 10937
word it_ponte
half of voc

...

word it_congresso
half of vocab 17689
word it_alonso
half of vocab 17690
word it_derivare
half of vocab 17691
word it_camerun
half of vocab 17692
word it_monica
half of vocab 17693
word it_carpi
half of vocab 17694
word it_baseball
half of vocab 17695
word it_comporre
half of vocab 17696
word it_garantito
half of vocab 17697
word it_analitica
half of vocab 17698
word it_fondatore
half of vocab 17699
word it_alternativa
half of vocab 17700
word it_delfino
half of vocab 17701
word it_sistematicamente
half of vocab 17702
word it_tortura
half of vocab 17703
word it_thriller
half of vocab 17704
word it_misero
half of vocab 17705
word it_autonomia
half of vocab 17706
word it_nel
half of vocab 17707
word it_bulgaro
half of vocab 17708
word it_diplomatico
half of vocab 17709
word it_debito
half of vocab 17710
word it_costruzione
half of vocab 17711
word it_bulgari
half of vocab 17712
word it_alive
half of vocab 17713
word it_migrazione
half of vocab 17714
word it_esemplare
half of vocab 17715


word it_corpo
half of vocab 17914
word it_enea
half of vocab 17915
word it_colture
half of vocab 17916
word it_specchi
half of vocab 17917
word it_seri
half of vocab 17918
word it_intelligenza
half of vocab 17919
word it_coltura
half of vocab 17920
word it_angela
half of vocab 17921
word it_femminile
half of vocab 17922
word it_venus
half of vocab 17923
word it_elisa
half of vocab 17924
word it_angelo
half of vocab 17925
word it_casa
half of vocab 17926
word it_esecutivo
half of vocab 17927
word it_parentela
half of vocab 17928
word it_agrario
half of vocab 17929
word it_statuto
half of vocab 17930
word it_agraria
half of vocab 17931
word it_legislatura
half of vocab 17932
word it_devastante
half of vocab 17933
word it_incisione
half of vocab 17934
word it_mammiferi
half of vocab 17935
word it_rubrica
half of vocab 17936
word it_generazioni
half of vocab 17937
word it_prostituzione
half of vocab 17938
word it_linux
half of vocab 17939
word it_agricoltori
half of vocab 17940
word it_inf

word it_pippo
half of vocab 18139
word it_certo
half of vocab 18140
word it_documentata
half of vocab 18141
word it_khmer
half of vocab 18142
word it_artiglieria
half of vocab 18143
word it_stimata
half of vocab 18144
word it_terziario
half of vocab 18145
word it_loggia
half of vocab 18146
word it_deserto
half of vocab 18147
word it_visconte
half of vocab 18148
word it_utile
half of vocab 18149
word it_visconti
half of vocab 18150
word it_modesto
half of vocab 18151
word it_isolamento
half of vocab 18152
word it_esercizi
half of vocab 18153
word it_dance
half of vocab 18154
word it_gigantesca
half of vocab 18155
word it_salvare
half of vocab 18156
word it_adriatico
half of vocab 18157
word it_marittimo
half of vocab 18158
word it_accumulo
half of vocab 18159
word it_soffitto
half of vocab 18160
word it_ghiaccio
half of vocab 18161
word it_nevada
half of vocab 18162
word it_indipendente
half of vocab 18163
word it_segmento
half of vocab 18164
word it_invasione
half of vocab 18165
word i

word it_tana
half of vocab 18363
word it_sottomarino
half of vocab 18364
word it_nintendo
half of vocab 18365
word it_incredibilmente
half of vocab 18366
word it_costante
half of vocab 18367
word it_sheffield
half of vocab 18368
word it_felicità
half of vocab 18369
word it_dotato
half of vocab 18370
word it_terni
half of vocab 18371
word it_stilo
half of vocab 18372
word it_intervento
half of vocab 18373
word it_colombo
half of vocab 18374
word it_essere
half of vocab 18375
word it_dotata
half of vocab 18376
word it_confessione
half of vocab 18377
word it_scienziati
half of vocab 18378
word it_pompei
half of vocab 18379
word it_indagine
half of vocab 18380
word it_eccezione
half of vocab 18381
word it_combinata
half of vocab 18382
word it_rana
half of vocab 18383
word it_confessioni
half of vocab 18384
word it_marcato
half of vocab 18385
word it_comandante
half of vocab 18386
word it_imprenditori
half of vocab 18387
word it_cerimonia
half of vocab 18388
word it_imprenditore
half of voc

{'it_migliora': 0}

In [ ]:
evaluate(self, source_lang, target_lang, gtt, sample, verbose=True)

# Method 2: Most Common Target Translation

### Initialize the model

In [46]:
from models import BiW2V_mle

# create model
model_2 = BiW2V_mle(bilingual_dict = translations,
                       vocab = vocab, H = EMBEDDING_SIZE)

# intialize TF graphs
model_2.BuildCoreGraph()
model_2.BuildTrainingGraph()
model_2.BuildValidationGraph()

... TF graph created for BiW2V model.
... TF graph created for BiW2V training.
... TF graph created for BiW2V validation.


### Training

In [50]:
# fresh data generator
DATA_GENERATOR = batch_generator(raw_data, vocab, BATCH_SIZE, WINDOW_SIZE, MAX_EPOCHS)

In [51]:
# train
start = time.time()
model_2.train(nBATCHES, DATA_GENERATOR, TEST_WORDS, learning_rate = ALPHA)
tot = (time.time() - start)
print('... {} batches trained in {} seconds'.format(nBATCHES, tot))

... Model Initialized
	 <tf.Variable 'Embedding_Layer/ContextEmbeddings:0' shape=(20003, 200) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/WordEmbeddings:0' shape=(20003, 200) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/b:0' shape=(20003,) dtype=float32_ref>
... Starting Training
... STEP 0 : Average Loss : 0.00252045059204
   [en_the] closest:  en_bern, en_mini, it_9, en_trout, en_gather, en_rival, en_homes, it_politica,
   [en_last] closest:  it_led, en_idaho, it_funzioni, en_up, it_svevia, it_trascrizione, en_climbing, it_foto,
   [it_si] closest:  en_6th, en_batting, en_reporter, en_shortened, it_alpini, it_tecnologie, en_addresses, en_credits,
   [it_suo] closest:  en_overhead, en_rico, en_venice, it_provenza, it_raven, it_battezzato, it_fire, it_normativa,
... STEP 5000 : Average Loss : 4.71148335021
... STEP 10000 : Average Loss : 4.22817595466
   [en_the] closest:  en_a, en_to, en_and, en_bern, en_mini, en_in, en_trout, it_politica,
   [en_last] closest:  it_led, en_idah

### Save the Embeddings.

In [52]:
# context 
filename = SAVE_TO + '/en_it_mle_50K_V_dec19.pkl'
with open(filename, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(model_2.context_embeddings, f, pickle.HIGHEST_PROTOCOL)

# word
filename = SAVE_TO + '/en_it_mle_50K_U_dec19.pkl'
with open(filename, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(model_2.word_embeddings, f, pickle.HIGHEST_PROTOCOL)

# Method 3: Closest Translation

### Initialize the model

In [56]:
from models import BiW2V_nn

# create model
model_3 = BiW2V_nn(bilingual_dict = translations,
                   vocab = vocab, H = EMBEDDING_SIZE)

# intialize TF graphs
model_3.BuildCoreGraph()
model_3.BuildTrainingGraph()
model_3.BuildValidationGraph()

... TF graph created for BiW2V model.
... TF graph created for BiW2V training.
... TF graph created for BiW2V validation.


### Training

In [57]:
# fresh data generator
DATA_GENERATOR = batch_generator(raw_data, vocab, BATCH_SIZE, WINDOW_SIZE, MAX_EPOCHS)

In [58]:
# train
nBATCHES = 5000 # Takes too long w/ nn so we'll only do 5K
start = time.time()
model_3.train(nBATCHES, DATA_GENERATOR, TEST_WORDS, learning_rate = ALPHA)
tot = (time.time() - start)
print('... {} batches trained in {} seconds'.format(nBATCHES, tot))

... Model Initialized
	 <tf.Variable 'Embedding_Layer/ContextEmbeddings:0' shape=(20003, 200) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/WordEmbeddings:0' shape=(20003, 200) dtype=float32_ref>
	 <tf.Variable 'Hidden_Layer/b:0' shape=(20003,) dtype=float32_ref>
... Starting Training
... STEP 0 : Average Loss : 0.02340858078
   [en_the] closest:  en_nutrition, it_ma, en_prolonged, it_falco, it_vercelli, en_artists, en_beliefs, it_idraulico,
   [en_last] closest:  en_earn, en_renowned, it_decadimento, it_boy, en_korea, en_basement, en_resulting, en_told,
   [it_si] closest:  en_qualities, it_generato, it_scuole, en_employer, en_situation, en_chorus, en_to, en_inadequate,
   [it_suo] closest:  en_sat, it_rimpiazzare, en_playing, it_triangolare, it_realmente, en_profile, it_tutta, it_stirpe,
... STEP 500 : Average Loss : 6.30888293695
... STEP 1000 : Average Loss : 5.52120716763
   [en_the] closest:  en_nutrition, en_artists, it_falco, it_ma, en_beliefs, it_saturno, it_anime, it_siccit

### Save the Embeddings.

In [59]:
# context 
filename = SAVE_TO + '/en_it_nn_5K_V_dec19.pkl'
with open(filename, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(model_2.context_embeddings, f, pickle.HIGHEST_PROTOCOL)

# word
filename = SAVE_TO + '/en_it_nn_5K_U_dec19.pkl'
with open(filename, 'wb') as f:
    # Pickle the 'data' dictionary using the highest protocol available.
    pickle.dump(model_2.word_embeddings, f, pickle.HIGHEST_PROTOCOL)